In [1]:
import pandas as pd
import numpy as np



In [2]:
participants = ['VG_01', 'VG_02']
n_participants = len(participants)
conditions = list(range(1,4))
n_conditions = len(conditions)

participant_entries = participants*n_conditions
condition_entries = conditions*n_participants

feature1_entries = np.round(np.random.random(n_participants*n_conditions), 2)
feature1_entries_alt = np.round(np.random.random(n_participants*n_conditions), 2)
feature2_entries = np.round(np.random.random(n_participants*n_conditions), 2)
feature3_entries = np.round(np.random.random(n_participants*n_conditions), 2)


df1 = pd.DataFrame(

    {

        "participant": participant_entries,

        "condition": condition_entries,

        "f1": feature1_entries,

        "f2": feature2_entries,

    },


)
display(df1)
df4 = pd.DataFrame(

    {


        "participant": participant_entries,

        "condition": condition_entries,

        "f1": feature1_entries_alt,

        "f3": feature2_entries,

    },


)
display(df4)

merge_on = [ 'participant', 'condition']
preserved_left_cols = [ c for c in df1.columns if (c in merge_on) or (not c in df4.columns) ]
df1 = df1.loc[:,preserved_left_cols]
display(df1)
result = pd.merge(df1, df4, how='outer')
result

,participant,condition,f1,f2
0,VG_01,1,0.68,0.97
1,VG_02,2,0.37,0.94
2,VG_01,3,0.25,0.22
3,VG_02,1,0.45,0.66
4,VG_01,2,0.33,0.45
5,VG_02,3,0.49,0.54


,participant,condition,f1,f3
0,VG_01,1,0.01,0.97
1,VG_02,2,0.29,0.94
2,VG_01,3,0.09,0.22
3,VG_02,1,0.84,0.66
4,VG_01,2,0.42,0.45
5,VG_02,3,0.18,0.54


,participant,condition,f2
0,VG_01,1,0.97
1,VG_02,2,0.94
2,VG_01,3,0.22
3,VG_02,1,0.66
4,VG_01,2,0.45
5,VG_02,3,0.54


,participant,condition,f2,f1,f3
0,VG_01,1,0.97,0.01,0.97
1,VG_02,2,0.94,0.29,0.94
2,VG_01,3,0.22,0.09,0.22
3,VG_02,1,0.66,0.84,0.66
4,VG_01,2,0.45,0.42,0.45
5,VG_02,3,0.54,0.18,0.54


In [29]:
conds_per_name = {'A': 2, 'B': 2}
name_cond_counts = { name: { i: np.random.randint(2,5) for i in range(names[name])} for name in conds_per_name }
print(f"name_conds = {name_conds}")
K = 4

# data_by_participant = {name: (np.random.random((rep, K)), np.random.randint(1,5,size=rep))
#         for name in names.items()}
print(f"[cond_counts for cond_counts in name_cond_counts.items()] = {[cond_counts for name, cond_counts in name_cond_counts.items()]}")
conds_by_participant = {name: (np.repeat(list(cond_counts.keys()), list(cond_counts.values())))
        for name, cond_counts in name_cond_counts.items()}
data_by_participant = {name: (np.random.random((part_conds.size, K)), part_conds)
        for name, part_conds in conds_by_participant.items()}
window_columns = [f'v{t}' for t in range(K)]
data_by_participant

name_conds = {'A': {0: 2, 1: 2}, 'B': {0: 4, 1: 3}}
[cond_counts for cond_counts in name_cond_counts.items()] = [{0: 2, 1: 3}, {0: 3, 1: 4}]


{'A': (array([[0.84043961, 0.71108416, 0.34657436, 0.32329678],
         [0.17361658, 0.03665855, 0.60878803, 0.10196465],
         [0.68303651, 0.37591534, 0.05029533, 0.75669384],
         [0.45173213, 0.51971521, 0.16052221, 0.37213022],
         [0.61796894, 0.53478307, 0.30292712, 0.3264995 ]]),
  array([0, 0, 1, 1, 1])),
 'B': (array([[0.20892372, 0.09986375, 0.90896521, 0.47980956],
         [0.28779061, 0.16889672, 0.87190814, 0.63162104],
         [0.44031074, 0.4866496 , 0.02923739, 0.84364973],
         [0.83446995, 0.52123696, 0.01187665, 0.87978511],
         [0.13319783, 0.18116142, 0.60039256, 0.01289194],
         [0.30563316, 0.38208026, 0.72417247, 0.07373143],
         [0.12432608, 0.90891349, 0.337435  , 0.94385223]]),
  array([0, 0, 0, 1, 1, 1, 1]))}

In [30]:
cond_counts = name_cond_counts['A']
np.repeat(list(cond_counts.keys()), list(cond_counts.values()))

array([0, 0, 1, 1, 1])

In [53]:
windows_np, condition_labels_np = map(np.concatenate, zip(*data_by_participant.values()))
windows = pd.DataFrame(windows_np, columns=window_columns)

window_counts = {prt: len(cnds) for prt,(wndws,cnds) in data_by_participant.items()}
# {'A': 3, 'B': 3, 'C': 4}
# participant_labels = pd.Series(
#     np.repeat(list(window_counts), list(window_counts.values())), name='participant')
# condition_labels = pd.Series(condition_labels_np, name='condition')
participant_labels_np = np.repeat(list(window_counts), list(window_counts.values()))
participant_condition_labels = pd.DataFrame(
    np.vstack((participant_labels_np, condition_labels_np)).T,
    columns=('participant','condition') )

# ['A', 'A', 'A', 'B', 'B', 'B', 'C', 'C', 'C', 'C']

window_indices = participant_condition_labels.groupby(["participant", "condition"]).cumcount().rename('window index')
# out = pd.concat(
#     [participant_labels, condition_labels, window_indices, windows],
#     axis=1)


out = pd.concat(
    [participant_condition_labels, window_indices, windows],
    axis=1)
out

,participant,condition,window index,v0,v1,v2,v3
0,A,0,0,0.840440,0.711084,0.346574,0.323297
1,A,0,1,0.173617,0.036659,0.608788,0.101965
2,A,1,0,0.683037,0.375915,0.050295,0.756694
3,A,1,1,0.451732,0.519715,0.160522,0.372130
4,A,1,2,0.617969,0.534783,0.302927,0.326500
5,B,0,0,0.208924,0.099864,0.908965,0.479810
6,B,0,1,0.287791,0.168897,0.871908,0.631621
7,B,0,2,0.440311,0.486650,0.029237,0.843650
8,B,1,0,0.834470,0.521237,0.011877,0.879785
9,B,1,1,0.133198,0.181161,0.600393,0.012892


0     0
1     1
2     0
3     1
4     2
5     0
6     1
7     2
8     0
9     1
10    2
11    3
Name: window index, dtype: int64

In [ ]:
out


In [ ]:
a = np.array(s)
np.random.shuffle(a)

shuf_s = pd.Series(a, name='name')
counts = pd.Series(shuf_s.groupby(shuf_s).cumcount(), name='repetition')

In [ ]:
pd.concat((shuf_s,counts),axis=1)

In [ ]:
s.name

In [ ]:
shuf_s.name

In [ ]:
counts = list(names.values())
np.concatenate(
            [np.arange(count) for count in counts ])